# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Pipelines service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following activites:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Creating Model steps - Create a model for use in transform steps or later publication as an endpoint.
* Parametrized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

## Layout of the SageMaker ModelBuild Project Template

The template provides a starting point for bringing your SageMaker Pipeline development to production.

```
sagemaker-project-modelbuild/
|-- codebuild-buildspec.yml
|-- CONTRIBUTING.md
|-- pipelines
|   |-- dsoaws
|   |   |-- evaluate.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess.py
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
|-- README.md
|-- setup.cfg
|-- setup.py
|-- tests
|   `-- test_pipelines.py
`-- tox.ini
```

A description of some of the artifacts is provided below:
<br/><br/>
Your codebuild execution instructions:
```
sagemaker-project-modelbuild/
|-- codebuild-buildspec.yml
```
<br/><br/>
Your pipeline artifacts, which includes a pipeline module defining the required `get_pipeline` method that returns an instance of a SageMaker pipeline, a preprocessing script that is used in feature engineering, and a model evaluation script to measure the accuracy of the trained model:

```
|-- pipelines
|   |-- dsoaws
|   |   |-- evaluate_metrics.py
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |   `-- preprocess-scikit-text-to-bert-feature-store.py
```
<br/><br/>
Utility modules for getting pipeline definition jsons and running pipelines:

```
|-- pipelines
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
```
<br/><br/>
Python package artifacts:

```
|-- setup.cfg
|-- setup.py
```
<br/><br/>
A stubbed testing module for testing your pipeline as you develop:

```
|-- tests
|   `-- test_pipelines.py
```
<br/><br/>
The `tox` testing framework configuration:

```
`-- tox.ini
```

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

![A typical ML Application pipeline](../img/pipeline-full.png)

### Getting some constants

We get some constants from the local execution environment.

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
default_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
print(role)

In [ ]:
%store -r sagemaker_project_name
%store -r sagemaker_project_name_and_id
%store -r sagemaker_mlops_build_code

In [ ]:
print('Project name: {}'.format(sagemaker_project_name))
print('Project name and id: {}'.format(sagemaker_project_name_and_id))
print('Source code for our mlops build pipeline: {}'.format(sagemaker_mlops_build_code))

# Get a Pipeline using our Cloned Repository

Here we get the pipeline instance from your pipeline module so that we can work with it.

In [ ]:
import sys

sys.path.insert(0, sagemaker_mlops_build_code)

In [ ]:
########
## NOTE:  IF YOU MAKE CHANGES TO get_pipeline, 
#         YOU NEED TO MAKE THE CHANGE TO THE CLONED REPO
#         AT `sagemaker_mlops_build_code`
########

from pipelines.dsoaws.pipeline import get_pipeline

pipeline = get_pipeline(
    pipeline_name=sagemaker_project_name_and_id,    
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=sagemaker_project_name_and_id,
    base_job_prefix=sagemaker_project_name_and_id
)

print(pipeline)

In [ ]:
pipeline.definition()

# AT THIS POINT, YOUR PIPELINE IS ALREADY EXECUTING
If you run the following code, you will start ANOTHER pipeline execution.

# Start Pipeline Execution

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [ ]:
raw_input_data_s3_uri = 's3://{}/amazon-reviews-pds/tsv/'.format(default_bucket)
print(raw_input_data_s3_uri)

# IGNORE THE FOLLOWING ERROR:
```
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
```

In [ ]:
print(pipeline)

execution = pipeline.start(
    parameters=dict(
        InputDataUrl=raw_input_data_s3_uri,
        ProcessingInstanceCount=1,
        ProcessingInstanceType='ml.c5.2xlarge',
        MaxSeqLength=64,
        BalanceDataset='True',
        TrainSplitPercentage=0.9,
        ValidationSplitPercentage=0.05,
        TestSplitPercentage=0.05,
#        FeatureStoreOfflinePrefix='', # TODO:  pass this in or leave empty for default?
#        FeatureGroupName='', # TODO:  pass this in or leave empty for default?
        TrainingInstanceType='ml.c5.9xlarge',
        TrainingInstanceCount=1,
        Epochs=1,
        LearningRate=0.00001,
        Epsilon=0.00000001,
        TrainBatchSize=128,
        ValidationBatchSize=128,
        TestBatchSize=128,
        TrainStepsPerEpoch=50,
        ValidationSteps=50,
        TestSteps=50,
        TrainVolumeSize=1024,
        UseXLA='True',
        UseAMP='True',
        FreezeBERTLayer='False',
        EnableSageMakerDebugger='False',
        EnableCheckpointing='False',
        EnableTensorboard='False',
        InputMode='File',
        RunValidation='True',
        RunTest='Fasle',
        RunSamplePredictions='False', 
        MinAccuracyValue=0.01,
        ModelApprovalStatus='Approved', # TODO:  Figure out why this is not working
        DeployInstanceType='ml.m5.4xlarge',
        DeployInstanceCount=1
    )
)

In [ ]:
execution_run = execution.describe()
print(execution_run)

In [ ]:
execution_run_name = execution_run['PipelineExecutionDisplayName']
print(execution_run_name)

# List Execution Steps

In [ ]:
execution.list_steps()

In [ ]:
%%time
execution.wait()

In [ ]:
execution.list_steps()

# List All Artifacts Generated By The Pipeline

In [ ]:
processing_job_name=None
training_job_name=None

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step['StepName'] == 'Processing':
        processing_job_name=execution_step['Metadata']['ProcessingJob']['Arn'].split('/')[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step['StepName'] == 'Train':
        training_job_name=execution_step['Metadata']['TrainingJob']['Arn'].split('/')[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# Create the Experiment 

We already create the Experiment in run_pipeline.py.

In [ ]:
# import time
# from smexperiments.experiment import Experiment

# timestamp = int(time.time())

# experiment = Experiment.create(
#                 experiment_name=pipeline_name,
#                 description='Amazon Customer Reviews BERT Pipeline Experiment', 
#                 sagemaker_boto_client=sm)

# experiment_name = experiment.experiment_name
# print('Experiment name: {}'.format(experiment_name))

# Associate the Pipeline Execution with our Experiment

In [ ]:
import time
from smexperiments.trial import Trial

trial = Trial.create(trial_name=execution_run_name,
                     experiment_name=pipeline_name,
                     sagemaker_boto_client=sm)

trial_name = trial.trial_name
print('Trial name: {}'.format(trial_name))

# Add Processing Job and Training Job to our Experiment

In [ ]:
# -aws-processing-job is the default name assigned by ProcessingJob
processing_job_tc = '{}-aws-processing-job'.format(processing_job_name)

print(processing_job_tc)

In [ ]:
response = sm.associate_trial_component(
    TrialComponentName=processing_job_tc,
    TrialName=trial_name
)

In [ ]:
# -aws-training-job is the default name assigned by TrainingJob
training_job_tc = '{}-aws-training-job'.format(training_job_name)

print(training_job_tc)

In [ ]:
response = sm.associate_trial_component(
    TrialComponentName=training_job_tc,
    TrialName=trial_name
)

# Log Additional Parameters within Trial

In [ ]:
from smexperiments import tracker

processing_job_tracker = tracker.Tracker.load(trial_component_name=processing_job_tc)
print(processing_job_tracker)

In [ ]:
processing_job_tracker.log_parameters({
    "balance_dataset": str(balance_dataset), 
})

# must save after logging
processing_job_tracker.trial_component.save()

# Analyze Experiment

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

import pandas as pd
pd.set_option("max_colwidth", 500)
#pd.set_option("max_rows", 100)

experiment_analytics = ExperimentAnalytics(
    experiment_name=pipeline_name,
)

experiment_analytics_df = experiment_analytics.dataframe()
experiment_analytics_df

# Download the Trained Model
Fix these parameters using the sagemaker_project_name_and_id

In [ ]:
training_job_arn=None

for execution_step in execution.list_steps():
    if execution_step["StepName"] == "TrainModel":
        training_job_arn = execution_step["Metadata"]["TrainingJob"]["Arn"]
        break

print(training_job_arn)

In [ ]:
training_job_name = training_job_arn.split('/')[-1]
print(training_job_name)

In [ ]:
!aws s3 cp s3://$default_bucket/$sagemaker_project_name_and_id/output/model/$training_job_name/output/model.tar.gz ./
    

In [ ]:
!tar -C ./downloaded_model -zxvf model.tar.gz

In [ ]:
!ls -al ./downloaded_model

# Approve the Registered Model for Staging
The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

Notes:  
* You can do this within SageMaker Studio, as well.  However, we are approving programmatically here in this example.
* This approval is only for Staging.  For Production, you must go through the CodePipeline (deep link is below.)


In [ ]:
model_package_arn=None

for execution_step in execution.list_steps():
    if execution_step["StepName"] == "RegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break

print(model_package_arn)

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",
)

print(model_package_update_response)

# Check CodePipeline Deployment

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Check Deployment in <a target="blank" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-{}-modeldeploy/view?region={}">CodePipeline</a></b>'.format(sagemaker_project_name_and_id, region)))

# Check the Staging Endpoint

In [ ]:
staging_endpoint_name='{}-staging'.format(sagemaker_project_name)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker Staging REST Endpoint</a></b>'.format(region, staging_endpoint_name)))

# Run Predictions

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
default_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# _Wait Until the Staging Endpoint is Deployed_


In [ ]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=staging_endpoint_name)

# _Wait Until the ^^ Staging Endpoint ^^ is Deployed_

# Run A Sample Prediction in Staging

In [ ]:
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name=staging_endpoint_name,
                                sagemaker_session=sess,
                                model_name='saved_model',
                                model_version=0)

In [ ]:
reviews = ["This is not great!"]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

# Approve Staging Model for Production
![](../img/approve-model-staging-to-production.png)


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Approve Model for Production in <a target="blank" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-{}-modeldeploy/view?region={}">CodePipeline</a></b>'.format(sagemaker_project_name_and_id, region)))

# Get the Production Endpoint Name

In [ ]:
production_endpoint_name='{}-prod'.format(sagemaker_project_name)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker Production REST Endpoint</a></b>'.format(region, production_endpoint_name)))


# _Wait Until the Production Endpoint is Deployed_

In [ ]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=production_endpoint_name)

# _Wait Until the ^^ Production Endpoint ^^ is Deployed_

In [ ]:
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name=production_endpoint_name,
                                sagemaker_session=sess,
                                model_name='saved_model',
                                model_version=0)

In [ ]:
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}